In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import os
import requests
from dotenv import load_dotenv


load_dotenv()

True

In [4]:
province_data = pd.read_json('hook/data/api_province_with_amphure_tambon.json')
province_data

,id,name_th,name_en,geography_id,created_at,updated_at,deleted_at,amphure
0,1,กรุงเทพมหานคร,Bangkok,2,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 1001, 'name_th': 'เขตพระนคร', 'name_en..."
1,2,สมุทรปราการ,Samut Prakan,2,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 1101, 'name_th': 'เมืองสมุทรปราการ', '..."
2,3,นนทบุรี,Nonthaburi,2,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 1201, 'name_th': 'เมืองนนทบุรี', 'name..."
3,4,ปทุมธานี,Pathum Thani,2,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 1301, 'name_th': 'เมืองปทุมธานี', 'nam..."
4,5,พระนครศรีอยุธยา,Phra Nakhon Si Ayutthaya,2,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 1401, 'name_th': 'พระนครศรีอยุธยา', 'n..."
...,...,...,...,...,...,...,...,...
72,73,พัทลุง,Phatthalung,6,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 9301, 'name_th': 'เมืองพัทลุง', 'name_..."
73,74,ปัตตานี,Pattani,6,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 9401, 'name_th': 'เมืองปัตตานี', 'name..."
74,75,ยะลา,Yala,6,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 9501, 'name_th': 'เมืองยะลา', 'name_en..."
75,76,นราธิวาส,Narathiwat,6,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 9601, 'name_th': 'เมืองนราธิวาส', 'nam..."


In [19]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Get the API key from the environment variables
api_key = os.getenv('TAT_API_KEY')

# Define the base URL for the Tourism Authority of Thailand API
BASE_URL = "https://api.tourismauthorityofthailand.com"

# Create a session
session = requests.Session()

# Configure retries and backoff strategy
retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

# Fetch data with custom session
def fetch_recommend_list(api_key: str, days: int, geolocation: str, region: str):
    url = f"{BASE_URL}/routes"
    
    print(f">>>> {url}")
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Accept-Language": "th"
    }
    # params = {
    #     # "numberofday": days,
    #     # "geolocation": geolocation,
    #     # "region": region
    # }

    # try:
    response = session.get(url, headers=headers)
    #     response.raise_for_status()
    data = response.json().get("data", [])
    #     return data
    # except requests.exceptions.HTTPError as http_err:
    #     print(f"HTTP error occurred: {http_err}")
    # except Exception as err:
    #     print(f"An error occurred: {err}")

# Fetch recommended provinces
recommended_provinces = fetch_recommend_list(api_key, 2, "13.67,100.76", "C")



>>>> https://api.tourismauthorityofthailand.com/routes


KeyboardInterrupt: 

In [ ]:
# Convert the data to a DataFrame
if recommended_provinces:
    df = pd.DataFrame(recommended_provinces)

    # Display the DataFrame
    print(df.head())

    # Visualize the data (e.g., number of attractions per province)
    if not df.empty:
        plt.figure(figsize=(10, 6))
        df['province_name'] = df['province_name'].astype(str)
        df['province_name'].value_counts().plot(kind='bar', color='skyblue')
        plt.title('Number of Attractions per Province')
        plt.xlabel('Province')
        plt.ylabel('Number of Attractions')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
else:
    print("No data available. Please check your API key or parameters.")